In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

In [13]:
BATCH_SIZE = 64
MIN_REPLAY_MEMORY_SIZE = 100
GAMMA = 0.99

class DQNAgent:
  def __init__(self):

    #policy model
    # train every steps
    self.model = self.create_model()

    #target model
    #.predict() every steps
    self.target_model = self.create_model()
    self.target_model.set_weights(self.model.get_weights())

    self.replay_memory = ReplayMemory(size)
    self.target_update_counter = 0


  def create_model(self):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim= env.state))
    model.add(Dense(env.actions_space, activation="softmax"))
    model.compile(optimizer='adam', loss='mae')
    return model

  def update_replay_memory(self, experience):
    self.replay_memory.push(experience)
  
  def get_Q_value(self, state, step):
    return self.model.predict(np.array(state))

  def train(self, state, step):
    # Si il n'y a pas de memory
    if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
      return

    #checker si il ya asser d'experience fournir un batch
    if self.replay_memory.get_provide_sample(self, BATCH_SIZE):
      minibatch = self.replay_memory.get_sample(BATCH_SIZE)

      ##-------Recuperer en Avance tous les Q value------###

      #recuperer tous les state_t dans le batch
      state_t = np.array([experience[0] for experience in minibatch])
      #Creer une liste de prediction avec le Policy model pour tous les state_t
      Q_t = self.model.predict(state_t)

      #recuperer tous les state_t+1 dans le batch
      state_t_next = np.array([experience[2] for experience in minibatch])
      #Creer une liste de prediction avec le Target model pour tous les state_t
      Q_t_next = self.target_model.predict(state_t_next)
    
    X = []
    y = []

    ##--------Creer X et Y pour l'entrainement du model-----##
    for i, (state_t, action, reward, state_t_next) in enumerate(minibatch):
      #tant que c'est pas la dernier q value
      #calculer la q value actuel avec la belman equation
      if i < BATCH_SIZE:
        max_Q_t_next = np.max(Q_t_next[i])
        new_Q_t = reward + GAMMA * max_Q_t_next
      
      #si c'est la dernier q value ou l'assigne on reward
      else :
        new_Q_t = reward

      Q_value = Q_t[i]
      Q_value[action] = new_Q_t

      X.append(state_t)
      y.append(Q_value)

    self.model.fit(np.array(X), np.array(y), batch_size=BATCH_SIZE, verbose=1, shuffle=False) 

    
    ''' Finir
    if None :#L'episode est fini :
      self.target_update_counter += 1

    # If counter reaches set value, update target network with weights of main network
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

    # Queries main network for Q values given current observation space (environment state)
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, *state.shape)/255)[0]
    '''
https://pythonprogramming.net/training-deep-q-learning-dqn-reinforcement-learning-python-tutorial/

agent = DQNAgent()

SyntaxError: ignored

In [ ]:
from collections import namedtuple
Experience = namedtuple(
	'Experience', ('state', 'action', 'next_state', 'reward'))


In [17]:
e = Experience(1, 4, 2, 11)
e1 = Experience(2, 4, 2, 11)
e2 = Experience(3, 4, 2, 11)
e3 = Experience(4, 4, 2, 11)
e4 = Experience(5, 4, 2, 11)

minibatch = list()
minibatch.extend([e, e1, e2, e3, e4])

for index, (state_t, action, reward, state_t_next) in enumerate(minibatch) :
  print(index, state_t, action, reward, state_t_next)

0 1 4 2 11
1 2 4 2 11
2 3 4 2 11
3 4 4 2 11
4 5 4 2 11
